In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

import os

In [2]:
num_classes = 10
img_rows , img_cols , img_ch = 28 , 28 ,1 
input_shape = (img_rows , img_cols , img_ch)

In [3]:
(x_train , y_train ) ,(x_test , y_test )  = tf.keras.datasets.mnist.load_data()

In [4]:
x_train = x_train/ 255.0
x_test  = x_test / 255.0


x_train = x_train.reshape(x_train.shape[0], *input_shape)
x_test = x_test.reshape(x_test.shape[0], *input_shape)

In [5]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (Input, Activation, Dense, Flatten, Conv2D, 
                                     MaxPooling2D, Dropout, BatchNormalization)

epochs      = 200
batch_size  = 32

In [6]:
@tf.function
def con_layer(x , kernels , bias , s ):
    z = tf.nn.conv2d( x , kernels , strides=[1 , s , s, 1] , padding='VALID')
    z = tf.nn.relu(z + bias )
    return z

In [7]:
class SimpleConv_layer(tf.keras.layers.Layer):
    
    def __init__(self, num_kernals = 32 , kernal_size = (3 , 3 ) , stride= 1):
        
        super().__init__()
        self.num_kernals = num_kernals
        self.kernal_size = kernal_size
        self.stride = stride
        
    def build( self, inputs ):
        num_channels = inputs[-1]
        kernals_shape = (*self.kernal_size , num_channels  , self.num_kernals)
        
        glorot_init = tf.initializers.GlorotUniform()
        self.kernels = self.add_weight(name= 'kernal',
                                      shape = kernals_shape ,
                                      initializer = glorot_init ,
                                      trainable = True)
        self.bias = self.add_weight(name = 'bias',
                                   shape = (self.num_kernals,),
                                   initializer = 'random_normal',
                                   trainable = True)
        
    def call(self, inputs):
        return con_layer(inputs , self.kernels , self.bias , self.stride)
    
    def get_config(self):
        {
          'kernel_size': self.kernel_size,
            'strides': self.strides,
            'use_bias': self.use_bias
        }
        

In [8]:
from functools import partial
def l2_reg(coef=1e-2):
    """
    Returns a function computing a weighed L2 norm of a given tensor.
    (this is basically a reimplementation of f.keras.regularizers.l2())
    :param coef:    Weight for the norm
    :return:        Loss function
    """
    return lambda x: tf.reduce_sum(x ** 2) * coef

In [9]:
class ConvWithRegularizers(SimpleConv_layer):
    
    def __init__(self, num_kernels = 32 , 
                kernel_size = (3 ,3 ) ,
                stride =  1, 
                kernel_regularizer = l2_reg(),
                bias_regularizer = None ):
        super().__init__(num_kernels , kernel_size , stride)
        self.kernel_regularizer = kernel_regularizer
        self.bias_regularizer = bias_regularizer
        
    
    def build(self, input_shape):
        super().build(input_shape)
        if self.kernel_regularizer is not None:
            self.add_loss(partial(self.kernel_regularizer, self.kernels))
        if self.bias_regularizer is not None:
            self.add_loss(partial(self.bias_regularizer, self.bias))

In [10]:
conv = ConvWithRegularizers(num_kernels=32, kernel_size=(3, 3), stride=1,
                            kernel_regularizer=l2_reg(1.), bias_regularizer=l2_reg(1.))
conv.build(input_shape=tf.TensorShape((None, 28, 28, 1)))

In [11]:
print(conv.losses)

[<tf.Tensor: id=32, shape=(), dtype=float32, numpy=1.9072797>, <tf.Tensor: id=39, shape=(), dtype=float32, numpy=0.050915994>]


In [12]:
class LeNet5(Model):
    
    def __init__(self, num_classes , kernel_regularizer=l2_reg(), bias_regularizer=l2_reg()):
        super(LeNet5 , self ).__init__()
        
        self.conv1 = ConvWithRegularizers(
        6 , kernel_size = (5 ,5 ) , kernel_regularizer=kernel_regularizer , bias_regularizer=bias_regularizer)
        
        self.conv2 = ConvWithRegularizers(
        32 , kernel_size = (5,5) , kernel_regularizer=kernel_regularizer , bias_regularizer=bias_regularizer)
        
        
        self.max_pool = tf.keras.layers.MaxPool2D(pool_size=( 2,2 ))
        self.flatten  = tf.keras.layers.Flatten()
        
        self.hidden_layer1 = tf.keras.layers.Dense(120 , activation='relu')
        self.hidden_layer2 = tf.keras.layers.Dense(84 , activation='relu')
        self.out_put_layer = tf.keras.layers.Dense(num_classes , activation='softmax')
        
    def call(self, x ):
        x = self.conv1(x)
        x = self.max_pool(x)
        
        
        x = self.conv2(x)
        x= self.max_pool(x)
        
        x = self.flatten(x)
        x = self.hidden_layer1(x)
        x = self.hidden_layer2(x)
        return self.out_put_layer(x)
        

In [13]:
optimizer = tf.optimizers.SGD()
model = LeNet5(10, kernel_regularizer=l2_reg(), bias_regularizer=l2_reg())

In [14]:
dataset    = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)

In [16]:
batched_input_shape = tf.TensorShape((None, *input_shape))
model.build(input_shape=batched_input_shape)

In [17]:
model.summary()

Model: "le_net5_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_with_regularizers_3 (Co multiple                  156       
_________________________________________________________________
conv_with_regularizers_4 (Co multiple                  4832      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
flatten_1 (Flatten)          multiple                  0         
_________________________________________________________________
dense_3 (Dense)              multiple                  61560     
_________________________________________________________________
dense_4 (Dense)              multiple                  10164     
_________________________________________________________________
dense_5 (Dense)              multiple                  85

In [ ]:
log_string_template = 'Epoch {0:3}/{1}: main loss = {5}{2:5.3f}{6}; ' + \
                      'reg loss = {5}{3:5.3f}{6}; val acc = {5}{4:5.3f}%{6}'

def train_classifier_on_mnist(model, log_frequency=10):

    avg_main_loss = tf.keras.metrics.Mean(name='avg_main_loss', dtype=tf.float32)
    avg_reg_loss  = tf.keras.metrics.Mean(name='avg_reg_loss', dtype=tf.float32)

    #print("Training: {}start{}".format(log_begin_red, log_end_format))
    for epoch in range(epochs):
        for (batch_images, batch_gts) in dataset:    # For each batch of this epoch

            with tf.GradientTape() as grad_tape:     # Tell TF to tape the gradients
                y = model(batch_images)              # Feed forward
                main_loss = tf.losses.sparse_categorical_crossentropy(
                    batch_gts, y)                    # Compute loss
                reg_loss = sum(model.losses)         # List and add other losses
                loss = main_loss + reg_loss

            # Get the gradients of combined losses and back-propagate:
            grads = grad_tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

            # Keep track of losses for display:
            avg_main_loss.update_state(main_loss)
            avg_reg_loss.update_state(reg_loss)

        if epoch % log_frequency == 0 or epoch == (epochs - 1): # Log some metrics
            # Validate, computing the accuracy on test data:
            acc = tf.reduce_mean(tf.metrics.sparse_categorical_accuracy(
                tf.constant(y_test), model(x_test))).numpy() * 100

            main_loss = avg_main_loss.result()
            reg_loss = avg_reg_loss.result()

           # print(log_string_template.format(
                #epoch, epochs, main_loss, reg_loss, acc, log_begin_blue, log_end_format))

        avg_main_loss.reset_states()
        avg_reg_loss.reset_states()
    print("Training: {}end{}".format(log_begin_green, log_end_format))
    return model
    

model = LeNet5(10, kernel_regularizer=l2_reg(), bias_regularizer=l2_reg())
model = train_classifier_on_mnist(model, log_frequency=10)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

